In [ ]:
%matplotlib inline


# Robust OPT - Control variate - Sellar problem


In [ ]:
from __future__ import annotations

from gemseo import configure_logger
from gemseo.algos.design_space import DesignSpace
from gemseo.algos.parameter_space import ParameterSpace
from gemseo.disciplines.analytic import AnalyticDiscipline

from gemseo_umdo.scenarios.umdo_scenario import UMDOScenario

configure_logger()

Firstly,
we instantiate the disciplines of the Sellar problem:



In [ ]:
system = AnalyticDiscipline({
    "obj": "x**2 + z2 + y1**2 + exp(-y2)",
    "c1": "3.16 - y1 ** 2",
    "c2": "y2 - 24.0",
})
disc1 = AnalyticDiscipline({"y1": "(z1**2 + z2 + x - a*y2)**0.5"})
disc2 = AnalyticDiscipline({"y2": "2/10*log(1+exp(10*y1))-y1-2/10*log(2) + z1 + z2"})

as well as the design space:



In [ ]:
design_space = DesignSpace()
design_space.add_variable("x", 1, l_b=0.0, u_b=10.0, value=1.0)
design_space.add_variable("z1", 1, l_b=-10, u_b=10.0, value=4.0)
design_space.add_variable("z2", 1, l_b=0.0, u_b=10.0, value=3.0)

Secondly,
we define the uncertain space:



In [ ]:
uncertain_space = ParameterSpace()

with an uncertainty over the constant `"a"`:



In [ ]:
uncertain_space.add_random_variable(
    "a", "OTTriangularDistribution", minimum=0.1, maximum=0.3, mode=0.2
)

Then,
we build an [UMDOScenario][gemseo_umdo.scenarios.umdo_scenario.UMDOScenario]
to minimize a control variate-based estimation
of the expectation $\mathbb{E}[obj]$:



In [ ]:
scenario = UMDOScenario(
    [system, disc1, disc2],
    "MDF",
    "obj",
    design_space,
    uncertain_space,
    "Mean",
    statistic_estimation="ControlVariate",
    statistic_estimation_parameters={"n_samples": 100},
)

while satisfying margin constraints
of the form $\mathbb{E}[c_i]+3\mathbb{S}[c_i]$



In [ ]:
scenario.add_constraint("c1", "Margin", factor=3.0)
scenario.add_constraint("c2", "Margin", factor=3.0)

and execute it with a gradient-free optimizer:



In [ ]:
scenario.execute({"algo": "NLOPT_COBYLA", "max_iter": 200})

Lastly,
we can plot the optimization history view:



In [ ]:
scenario.post_process("OptHistoryView", save=True, show=False)